In [1]:
# https://qiita.com/segavvy/items/003e8b7e0f132f4fde1b

import re

class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface # 表層形(surface)
        self.base    = base    # 基本形(base)
        self.pos     = pos     # 品詞(pos)
        self.pos1    = pos1    # 品詞細分類1(pos1)
        
    def __str__(self):
        '''オブジェクトの文字列表現'''
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'\
            .format(self.surface, self.base, self.pos, self.pos1)

カラム	意味
1	先頭カラムは*。係り受け解析結果であることを示す。
2	文節番号（0から始まる整数）
3	係り先番号＋D
4	主辞/機能語の位置と任意の個数の素性列
5	係り関係のスコア。係りやすさの度合で、一般に大きな値ほど係りやすい。

In [2]:
class Chunk:
    def __init__(self):
        '''初期化'''
        self.morphs = []
        self.srcs   = []   # 係り元文節インデックス番号のリスト
        self.dst    = -1   # 係り先文節インデックス番号(初期値:-1, 係り先がない場合は-1のまま)
        
    def __str__(self):
        '''オブジェクトの文字列表現'''
        surface = ''
        for morph in self.morphs:
            surface += morph.surface
        return '{}\tsrcs{}\tdst[{}]'.format(surface, self.srcs, self.dst)

In [33]:
def neko_lines():
    with open('./neko.txt.cabocha3', encoding='utf-8') as neko_cabocha:
    
        chunks = dict()     # idxをkeyにChunkを格納
        
        for line in neko_cabocha:
            if line[:3] == 'EOS':
                
                # Chunkのリストを返す
                if len(chunks) > 0:

                    # chunksをkeyでソートし、valueのみ取り出し
                    sorted_tuple = sorted(chunks.items(), key=lambda x: x[0])
                    yield list(zip(*sorted_tuple))[1]  #[1]がリストのvalue部分
                    chunks.clear()

                else:
                    yield []
                    
            # 先頭が*の行は係り受け解析結果なので、Chunkを作成
            elif line[0] == '*':

                # Chunkのインデックス番号と係り先のインデックス番号取得
                cols = re.split('\s|D', line)
                idx = int(cols[1]) # Chunkのインデックス番号
                dst = int(cols[2]) # 係り先文節インデックス番号

                # Chunkを生成（なければ）し、係り先のインデックス番号セット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst

                # 係り先のChunkを生成（なければ）し、係り元インデックス番号追加
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx) # 係り元は複数あるのでappend
                    
            else:
                
                #タブとカンマで分割
                cols = re.split('\t|,', line)

                chunks[idx].morphs.append(Morph(
                        cols[0],    # 表層形(surface)
                        cols[7],    # 基本形(base)
                        cols[1],    # 品詞(pos)
                        cols[2]     # 品詞細分類1(pos1)
                    ))

        raise StopIteration

In [34]:
# 1文ずつリスト作成(iを1から開始するためにenumerate使用)
for i, chunks in enumerate(neko_lines(), 1):
    
    # 8文目を表示
    if i == 8:
        for j, chunk in enumerate(chunks):
            print('[{}]{}'.format(j, chunk))
        break

[0]吾輩は	srcs[]	dst[5]
[1]ここで	srcs[]	dst[2]
[2]始めて	srcs[1]	dst[3]
[3]人間という	srcs[2]	dst[4]
[4]ものを	srcs[3]	dst[5]
[5]見た。	srcs[0, 4]	dst[-1]
